In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e15/sample_submission.csv
/kaggle/input/playground-series-s3e15/data.csv


In [2]:
df = pd.read_csv("/kaggle/input/playground-series-s3e15/data.csv")

In [3]:
df.head()

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,0,Thompson,tube,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6
1,1,Thompson,tube,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2
2,2,Thompson,NaN,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5
3,3,Beus,annulus,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0
4,4,NaN,tube,13.79,686.0,NaN,11.1,11.1,457.0,2.8


In [4]:
df.describe()

,id,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
count,31644.000000,27192.000000,26853.000000,21229.000000,26156.000000,27055.000000,26885.000000,31644.000000
mean,15821.500000,10.640747,3068.011023,-0.000453,8.629255,14.174330,832.987391,3.796985
std,9134.980296,4.333683,1777.032080,0.100911,5.185692,19.838489,672.299239,1.983991
min,0.000000,0.100000,0.000000,-0.866700,1.000000,1.000000,10.000000,0.800000
25%,7910.750000,6.890000,1519.000000,-0.046600,5.000000,5.600000,318.000000,2.400000
50%,15821.500000,11.070000,2731.000000,0.003800,7.800000,10.000000,610.000000,3.400000
75%,23732.250000,13.790000,4069.000000,0.064800,10.800000,11.500000,914.000000,4.600000
max,31643.000000,20.680000,7975.000000,0.232000,37.500000,120.000000,3048.000000,19.300000


Simple Imputer

In [5]:
from sklearn.impute import SimpleImputer

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31644 entries, 0 to 31643
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   31644 non-null  int64  
 1   author               26620 non-null  object 
 2   geometry             26144 non-null  object 
 3   pressure [MPa]       27192 non-null  float64
 4   mass_flux [kg/m2-s]  26853 non-null  float64
 5   x_e_out [-]          21229 non-null  float64
 6   D_e [mm]             26156 non-null  float64
 7   D_h [mm]             27055 non-null  float64
 8   length [mm]          26885 non-null  float64
 9   chf_exp [MW/m2]      31644 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 2.4+ MB


In [7]:
numerical_cols = df.select_dtypes(include = ['float64']).columns.tolist()
categorical_cols = df.select_dtypes(include = ['object']).columns.tolist()

In [8]:
impute_columns = SimpleImputer(strategy = 'most_frequent')
df[categorical_cols] = impute_columns.fit_transform(df[categorical_cols])

In [9]:
df[categorical_cols]

,author,geometry
0,Thompson,tube
1,Thompson,tube
2,Thompson,tube
3,Beus,annulus
4,Thompson,tube
...,...,...
31639,Thompson,tube
31640,Thompson,tube
31641,Thompson,tube
31642,Thompson,tube


In [10]:
impute_columns = SimpleImputer(strategy = 'mean')
df[numerical_cols] = impute_columns.fit_transform(df[numerical_cols])

In [11]:
df[numerical_cols]

,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,7.000000,3770.000000,0.175400,8.629255,10.8,432.000000,3.6
1,10.640747,6049.000000,-0.041600,10.300000,10.3,762.000000,6.2
2,13.790000,2034.000000,0.033500,7.700000,7.7,457.000000,2.5
3,13.790000,3679.000000,-0.027900,5.600000,15.2,2134.000000,3.0
4,13.790000,686.000000,-0.000453,11.100000,11.1,457.000000,2.8
...,...,...,...,...,...,...,...
31639,10.640747,1736.000000,0.088600,8.629255,7.8,591.000000,2.3
31640,13.790000,3068.011023,-0.000453,4.700000,4.7,832.987391,3.9
31641,18.270000,658.000000,-0.122400,3.000000,3.0,150.000000,2.3
31642,6.890000,3825.000000,-0.000453,23.600000,23.6,1972.000000,3.7


In [12]:
df

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,0,Thompson,tube,7.000000,3770.000000,0.175400,8.629255,10.8,432.000000,3.6
1,1,Thompson,tube,10.640747,6049.000000,-0.041600,10.300000,10.3,762.000000,6.2
2,2,Thompson,tube,13.790000,2034.000000,0.033500,7.700000,7.7,457.000000,2.5
3,3,Beus,annulus,13.790000,3679.000000,-0.027900,5.600000,15.2,2134.000000,3.0
4,4,Thompson,tube,13.790000,686.000000,-0.000453,11.100000,11.1,457.000000,2.8
...,...,...,...,...,...,...,...,...,...,...
31639,31639,Thompson,tube,10.640747,1736.000000,0.088600,8.629255,7.8,591.000000,2.3
31640,31640,Thompson,tube,13.790000,3068.011023,-0.000453,4.700000,4.7,832.987391,3.9
31641,31641,Thompson,tube,18.270000,658.000000,-0.122400,3.000000,3.0,150.000000,2.3
31642,31642,Thompson,tube,6.890000,3825.000000,-0.000453,23.600000,23.6,1972.000000,3.7


In [13]:
submission = pd.DataFrame(df[['id','x_e_out [-]']])
submission.to_csv('submission.csv', index = False)